The goal of this is to create a web scraping program which scrapes all information on the webpage and turns it into a database. 

https://www.coloradoski.com/snow-report/

I would like the final product to resemble the following:

Name of Hill	Variable	Value	Day
Granby Ranch	24hr	0	March 15, 2025
Granby Ranch	48hr	5	March 15, 2025
Granby Ranch	Lifts Open	5/5	March 15, 2025
Granby Ranch	24hr	0	March 16, 2025
Granby Ranch	48hr	0	March 16, 2025
Granby Ranch	Lifts Open	5/5	March 16, 2025
Eldora	24hr	0	March 15, 2025
Eldora	48hr	3	March 15, 2025
Eldora	Lifts Open	10/10	March 15, 2025


# Step 1: Set up daily scrape...

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

KeyboardInterrupt: 

Set Up Extraction Variables

In [22]:
'''
name --> h3 class = "h5 text-left"<Arapahoe Basin</h3>
24hr --> div class = "value", <p>, <span class = "answer twentyfour">"</span>
48hr --> div class = "value", <p>, <span class = "answer fourtyeight">6"</span>
Mid-Mt Depth --> div class = "value", <p>, <span class = "answer mid-mtn">51"</span>
Surface Conditions --> div class = "value", <p>, <span class = "Surface">HT/HP</span>
Lifts Open --> this one is tricky as it needs to extract 3 values. Value 1: <div class = "value", <p class = "lifts-open">, <span class = "open">9</span>,<span class >/</span>, <span class = "total">9</span>
Green --> div class = "value", <p>, <span class = "green-runs">100%</span>
Blue --> div class = "value", <p>, <span class = "blue-runs">100%</span>
Black --> div class = "value", <p>, <span class = "diamond-runs">100%</span>
Double Black --> div class = "value", <p>, <span class = "double-diamond-runs">100%</span>

'''

'\nname --> h3 class = "h5 text-left"<Arapahoe Basin</h3>\n24hr --> div class = "value", <p>, <span class = "answer twentyfour">"</span>\n48hr --> div class = "value", <p>, <span class = "answer fourtyeight">6"</span>\nMid-Mt Depth --> div class = "value", <p>, <span class = "answer mid-mtn">51"</span>\nSurface Conditions --> div class = "value", <p>, <span class = "Surface">HT/HP</span>\nLifts Open --> this one is tricky as it needs to extract 3 values. Value 1: <div class = "value", <p class = "lifts-open">, <span class = "open">9</span>,<span class >/</span>, <span class = "total">9</span>\nGreen --> div class = "value", <p>, <span class = "green-runs">100%</span>\nBlue --> div class = "value", <p>, <span class = "blue-runs">100%</span>\nBlack --> div class = "value", <p>, <span class = "diamond-runs">100%</span>\nDouble Black --> div class = "value", <p>, <span class = "double-diamond-runs">100%</span>\n\n'

In [37]:
## Investigation into the HTML code so I can learn to extract from it... 

url = "https://www.coloradoski.com/snow-report/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Referer": "https://www.google.com/"
}

response = requests.get(url, headers=headers)

# Save raw HTML to a file
with open("snow_report_raw.html", "w", encoding="utf-8") as file:
    file.write(response.text)

print("HTML saved to snow_report_raw.html")

HTML saved to snow_report_raw.html


Scraping Below

In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Path to your ChromeDriver
driver_path = "C:/Users/danie/chromedriver-win64/chromedriver.exe"
csv_file_path = "C:/Users/danie/OneDrive/Desktop/PersonalProjects/SkiWebScraping/snow_report_data_soloradoski.csv"

# Set up the Chrome driver using Service
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the website
url = "https://www.coloradoski.com/snow-report/"
driver.get(url)

# Dynamic wait: Wait until at least one Mid-Mt Depth value is not "0"
try:
    WebDriverWait(driver, 30).until(
        lambda d: any(
            mid_mt.text.strip() != '0"'
            for mid_mt in d.find_elements(By.CLASS_NAME, "answer.mid-mtn")
        )
    )
except Exception as e:
    print("Timeout waiting for Mid-Mt Depth to load:", e)
    driver.quit()

# Get page source after JavaScript has loaded
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Close the browser
driver.quit()

# Initialize list to hold data
snow_data = []

# Extract resort data
resorts = soup.find_all("div", class_="inner")

for resort in resorts:
    # Extract Resort Name
    name_tag = resort.find("h3", class_="h5 text-left")
    
    # Extract Snow-related Data
    snow_24hr_tag = resort.find("span", class_="answer twentyfour")
    snow_48hr_tag = resort.find("span", class_="answer fortyeight")
    mid_mt_depth_tag = resort.find("span", class_="answer mid-mtn")
    surface_conditions_tag = resort.find("p", class_="surface")
    lifts_open_tag = resort.find("p", class_="lifts-open")
    green_runs_tag = resort.find("p", class_="green-runs")
    blue_runs_tag = resort.find("p", class_="blue-runs")
    black_runs_tag = resort.find("p", class_="diamond-runs")
    double_black_runs_tag = resort.find("p", class_="double-diamond-runs")

    # Ensure Resort Name exists
    if name_tag:
        name = name_tag.text.strip()
        snow_24hr = snow_24hr_tag.text.strip() if snow_24hr_tag else "N/A"
        snow_48hr = snow_48hr_tag.text.strip() if snow_48hr_tag else "N/A"
        mid_mt_depth = mid_mt_depth_tag.text.strip() if mid_mt_depth_tag else "N/A"
        surface_conditions = surface_conditions_tag.text.strip() if surface_conditions_tag else "N/A"
        lifts_open = lifts_open_tag.text.strip() if lifts_open_tag else "N/A"
        green_runs = green_runs_tag.text.strip() if green_runs_tag else "N/A"
        blue_runs = blue_runs_tag.text.strip() if blue_runs_tag else "N/A"
        black_runs = black_runs_tag.text.strip() if black_runs_tag else "N/A"
        double_black_runs = double_black_runs_tag.text.strip() if double_black_runs_tag else "N/A"

        # Append data
        snow_data.append({
            "Date": datetime.now().strftime("%Y-%m-%d"),
            "Resort": name,
            "Snow (24hr)": snow_24hr,
            "Snow (48hr)": snow_48hr,
            "Mid-Mt Depth": mid_mt_depth,
            "Surface Conditions": surface_conditions,
            "Lifts Open": lifts_open,
            "Green Runs": green_runs,
            "Blue Runs": blue_runs,
            "Black Runs": black_runs,
            "Double Black Runs": double_black_runs
        })

# Convert to DataFrame
new_data_df = pd.DataFrame(snow_data)

# Check if the CSV already exists
if os.path.exists(csv_file_path):
    # Read existing data
    existing_data_df = pd.read_csv(csv_file_path)
    
    # Append new data to existing data
    combined_df = pd.concat([existing_data_df, new_data_df], ignore_index=True)
else:
    # If CSV doesn't exist, the new data becomes the initial data
    combined_df = new_data_df

# Save the combined DataFrame back to CSV
combined_df.to_csv(csv_file_path, index=False)

# Display the newly added data
print(new_data_df)
new_data_df_coloradoski =  new_data_df


          Date             Resort Snow (24hr) Snow (48hr) Mid-Mt Depth  \
0   2025-02-09     Arapahoe Basin          2"          8"          52”   
1   2025-02-09    Aspen Highlands          7"         11"          41”   
2   2025-02-09     Aspen Mountain          9"         11"          37”   
3   2025-02-09         Buttermilk          6"          7"          30”   
4   2025-02-09             Cooper          3"          8"          37”   
5   2025-02-09    Copper Mountain          6"         10"          57”   
6   2025-02-09      Echo Mountain          0"          0"          18”   
7   2025-02-09             Eldora          6"          9"          30”   
8   2025-02-09       Granby Ranch          1"          5"          32”   
9   2025-02-09      Howelsen Hill          0"          0"          34”   
10  2025-02-09  Loveland Ski Area          4"          8"          44”   
11  2025-02-09            Monarch          0"          0"          41”   
12  2025-02-09         Powderhorn     

In [45]:
new_data_df_coloradoski =  new_data_df

In [44]:
new_data_df

,Date,Resort,Snow (24hr),Snow (48hr),Mid-Mt Depth,Surface Conditions,Lifts Open,Green Runs,Blue Runs,Black Runs,Double Black Runs
0,2025-02-09,Arapahoe Basin,"2""","8""",52”,HP/PP,9/9,100%,100%,100%,79%
1,2025-02-09,Aspen Highlands,"7""","11""",41”,PP,5/5,N/A,100%,100%,84%
2,2025-02-09,Aspen Mountain,"9""","11""",37”,PP,7/8,N/A,100%,100%,96%
3,2025-02-09,Buttermilk,"6""","7""",30”,PP,5/8,100%,100%,89%,0%
4,2025-02-09,Cooper,"3""","8""",37”,P/PP,5/5,100%,100%,100%,100%
5,2025-02-09,Copper Mountain,"6""","10""",57”,P,24/24,100%,100%,97%,100%
6,2025-02-09,Echo Mountain,"0""","0""",18”,MM,2/2,67%,100%,0%,0%
7,2025-02-09,Eldora,"6""","9""",30”,PP/HP,10/10,100%,97%,100%,90%
8,2025-02-09,Granby Ranch,"1""","5""",32”,PP,5/5,100%,100%,100%,N/A
9,2025-02-09,Howelsen Hill,"0""","0""",34”,MM/SP,3/3,100%,88%,100%,N/A


In [ ]:
Still missing: 
1. Beaver Creek
2. Breckenridge
3. Crested Butte
4. Keystone
5. Ski Granby Ranch
6. Silverton Mountain
7. Vail
8. Wolf Creek Ski Area

## Second Website

In [5]:
## Investigation into the HTML code so I can learn to extract from it... 

url = "https://www.onthesnow.com/colorado/skireport"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Referer": "https://www.google.com/"
}

response = requests.get(url, headers=headers)

# Save raw HTML to a file
with open("onthesnow_report_raw.html", "w", encoding="utf-8") as file:
    file.write(response.text)

print("HTML saved to snow_report_raw.html")

HTML saved to snow_report_raw.html


In [48]:
# Note: Takes the Higher Base Depth Value if a Range is Given
import os
import re
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# File paths
driver_path = r"C:\Users\danie\chromedriver-win64\chromedriver.exe"
csv_file_path = r"C:\Users\danie\OneDrive\Desktop\PersonalProjects\SkiWebScraping\snow_report_data_onthesnow.csv"
log_file_path = r"C:\Users\danie\OneDrive\Desktop\PersonalProjects\SkiWebScraping\scraper_log.txt"

# Set up logging
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.info("Script started.")

try:
    # Set up the Chrome driver using Service
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)

    # Open the website
    url = "https://www.onthesnow.com/colorado/skireport"
    driver.get(url)

    # Dynamic wait: Wait until resort rows are loaded
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "styles_row__HA9Yq"))
    )
    logging.info("Resort data loaded successfully.")

    # Get page source after JavaScript has loaded
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # Close the browser
    driver.quit()

    # Initialize list to hold data
    snow_data = []

    # Extract resort data
    resorts = soup.find_all("tr", class_="styles_row__HA9Yq")  # Each resort is in a table row

    for resort in resorts:
        data_tags = resort.find_all("span", class_="h4 styles_h4__x3zzi")

        # Ensure enough data points exist
        if len(data_tags) >= 5:
            open_trails_raw = data_tags[3].text.strip() if data_tags[3] else "N/A"

            # Regex to extract Trails Open, Total Trails, and Percentage Open
            match = re.search(r"(\d+)\s*/\s*(\d+)(\d{2,3})?\s*(\d+%)?", open_trails_raw)

            if match:
                trails_open = int(match.group(1))
                total_trails_raw = match.group(2)
                last_digits = int(match.group(3)) if match.group(3) else "N/A"
                percentage_open = match.group(4) if match.group(4) else "N/A"

                # Correction logic
                total_trails = int(total_trails_raw)
                corrected_total_trails = total_trails

                # Attempt to correct by removing trailing digits
                for i in range(1, 4):  # Check removing 1 to 3 digits
                    possible_total = int(str(total_trails)[:-i]) if len(str(total_trails)) > i else total_trails

                    if possible_total >= trails_open:
                        if percentage_open != "N/A":
                            calculated_percentage = round((trails_open / possible_total) * 100)
                            if calculated_percentage == int(percentage_open.strip('%')):
                                corrected_total_trails = possible_total
                                break
                        else:
                            # If percentage is not provided, use logical deduction
                            if possible_total - trails_open <= 50:  # Assuming it's reasonable
                                corrected_total_trails = possible_total
                                break

                # Final validation
                if corrected_total_trails < trails_open:
                    corrected_total_trails = "N/A"

            else:
                trails_open, total_trails_raw, corrected_total_trails, percentage_open = "N/A", "N/A", "N/A", "N/A"

            snow_data.append({
                "Date": datetime.now().strftime("%Y-%m-%d"),
                "Resort": data_tags[0].text.strip() if data_tags[0] else "N/A",          # Resort Name
                "Snow (24hr)": data_tags[1].text.strip().rstrip('"').rstrip('-') if data_tags[1] else "N/A",      # Cleaned Snow (24hr)
                "Base Depth": data_tags[2].text.strip() if data_tags[2] else "N/A",       # Base Depth
                "Trails Open": trails_open,                                               # Cleaned Trails Open
                "Total Trails": corrected_total_trails,                                   # Renamed Corrected Total Trails
                "Lifts Open": data_tags[4].text.strip().rstrip('-') if data_tags[4] else "N/A"        # Cleaned Lifts Open
            })

    # Convert to DataFrame
    new_data_df = pd.DataFrame(snow_data)

    # Separate Base Depth into two columns
    new_data_df[['Base Depth', 'Base Depth Notes']] = new_data_df['Base Depth'].str.extract(r'(\d+\"(?:-\d+\")?)(.*)')

    # Calculate % Trails Open
    new_data_df["% Trails Open"] = (new_data_df["Trails Open"] / new_data_df["Total Trails"] * 100).round(2)

    # Append or create CSV
    if os.path.exists(csv_file_path):
        existing_data_df = pd.read_csv(csv_file_path)
        combined_df = pd.concat([existing_data_df, new_data_df], ignore_index=True)
    else:
        combined_df = new_data_df

    # Save combined data
    combined_df.to_csv(csv_file_path, index=False)
    logging.info(f"Data successfully saved to {csv_file_path}")

    # Display newly added data
    print(new_data_df)

except Exception as e:
    logging.error(f"An error occurred: {e}")
    print(f"An error occurred: {e}")


          Date                         Resort Snow (24hr) Base Depth  \
0   2025-02-09        Arapahoe Basin Ski Area          2"        52"   
1   2025-02-09                 Aspen Snowmass          9"        53"   
2   2025-02-09                   Beaver Creek          2"        44"   
3   2025-02-09                   Breckenridge          7"        53"   
4   2025-02-09                         Cooper          3"        40"   
5   2025-02-09                Copper Mountain          6"        58"   
6   2025-02-09  Crested Butte Mountain Resort          0"        45"   
7   2025-02-09                  Echo Mountain          0"        30"   
8   2025-02-09         Eldora Mountain Resort          6"        30"   
9   2025-02-09                  Howelsen Hill          0"        34"   
10  2025-02-09                       Keystone          3"        43"   
11  2025-02-09              Loveland Ski Area          4"        46"   
12  2025-02-09               Monarch Mountain          0"       

In [ ]:
'''
First Script -- Colorado Ski
'''
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Path to your ChromeDriver
driver_path = "C:/Users/danie/chromedriver-win64/chromedriver.exe"
csv_file_path = "C:/Users/danie/OneDrive/Desktop/PersonalProjects/SkiWebScraping/snow_report_data_soloradoski.csv"

# Set up the Chrome driver using Service
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the website
url = "https://www.coloradoski.com/snow-report/"
driver.get(url)

# Dynamic wait: Wait until at least one Mid-Mt Depth value is not "0"
try:
    WebDriverWait(driver, 30).until(
        lambda d: any(
            mid_mt.text.strip() != '0"'
            for mid_mt in d.find_elements(By.CLASS_NAME, "answer.mid-mtn")
        )
    )
except Exception as e:
    print("Timeout waiting for Mid-Mt Depth to load:", e)
    driver.quit()

# Get page source after JavaScript has loaded
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Close the browser
driver.quit()

# Initialize list to hold data
snow_data = []

# Extract resort data
resorts = soup.find_all("div", class_="inner")

for resort in resorts:
    # Extract Resort Name
    name_tag = resort.find("h3", class_="h5 text-left")
    
    # Extract Snow-related Data
    snow_24hr_tag = resort.find("span", class_="answer twentyfour")
    snow_48hr_tag = resort.find("span", class_="answer fortyeight")
    mid_mt_depth_tag = resort.find("span", class_="answer mid-mtn")
    surface_conditions_tag = resort.find("p", class_="surface")
    lifts_open_tag = resort.find("p", class_="lifts-open")
    green_runs_tag = resort.find("p", class_="green-runs")
    blue_runs_tag = resort.find("p", class_="blue-runs")
    black_runs_tag = resort.find("p", class_="diamond-runs")
    double_black_runs_tag = resort.find("p", class_="double-diamond-runs")

    # Ensure Resort Name exists
    if name_tag:
        name = name_tag.text.strip()
        snow_24hr = snow_24hr_tag.text.strip() if snow_24hr_tag else "N/A"
        snow_48hr = snow_48hr_tag.text.strip() if snow_48hr_tag else "N/A"
        mid_mt_depth = mid_mt_depth_tag.text.strip() if mid_mt_depth_tag else "N/A"
        surface_conditions = surface_conditions_tag.text.strip() if surface_conditions_tag else "N/A"
        lifts_open = lifts_open_tag.text.strip() if lifts_open_tag else "N/A"
        green_runs = green_runs_tag.text.strip() if green_runs_tag else "N/A"
        blue_runs = blue_runs_tag.text.strip() if blue_runs_tag else "N/A"
        black_runs = black_runs_tag.text.strip() if black_runs_tag else "N/A"
        double_black_runs = double_black_runs_tag.text.strip() if double_black_runs_tag else "N/A"

        # Append data
        snow_data.append({
            "Date": datetime.now().strftime("%Y-%m-%d"),
            "Resort": name,
            "Snow (24hr)": snow_24hr,
            "Snow (48hr)": snow_48hr,
            "Mid-Mt Depth": mid_mt_depth,
            "Surface Conditions": surface_conditions,
            "Lifts Open": lifts_open,
            "Green Runs": green_runs,
            "Blue Runs": blue_runs,
            "Black Runs": black_runs,
            "Double Black Runs": double_black_runs
        })

# Convert to DataFrame
new_data_df = pd.DataFrame(snow_data)

# Check if the CSV already exists
if os.path.exists(csv_file_path):
    # Read existing data
    existing_data_df = pd.read_csv(csv_file_path)
    
    # Append new data to existing data
    combined_df = pd.concat([existing_data_df, new_data_df], ignore_index=True)
else:
    # If CSV doesn't exist, the new data becomes the initial data
    combined_df = new_data_df

# Save the combined DataFrame back to CSV
combined_df.to_csv(csv_file_path, index=False)

# Display the newly added data
print(new_data_df)


'''
Second Script -- OnTheSnow
'''
# Note: Takes the Higher Base Depth Value if a Range is Given
import os
import re
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# File paths
driver_path = r"C:\Users\danie\chromedriver-win64\chromedriver.exe"
csv_file_path = r"C:\Users\danie\OneDrive\Desktop\PersonalProjects\SkiWebScraping\snow_report_data_onthesnow.csv"
log_file_path = r"C:\Users\danie\OneDrive\Desktop\PersonalProjects\SkiWebScraping\scraper_log.txt"

# Set up logging
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.info("Script started.")

try:
    # Set up the Chrome driver using Service
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)

    # Open the website
    url = "https://www.onthesnow.com/colorado/skireport"
    driver.get(url)

    # Dynamic wait: Wait until resort rows are loaded
    WebDriverWait(driver, 40).until(
        EC.presence_of_element_located((By.CLASS_NAME, "styles_row__HA9Yq"))
    )
    logging.info("Resort data loaded successfully.")

    # Get page source after JavaScript has loaded
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # Close the browser
    driver.quit()

    # Initialize list to hold data
    snow_data = []

    # Extract resort data
    resorts = soup.find_all("tr", class_="styles_row__HA9Yq")  # Each resort is in a table row

    for resort in resorts:
        data_tags = resort.find_all("span", class_="h4 styles_h4__x3zzi")

        # Ensure enough data points exist
        if len(data_tags) >= 5:
            open_trails_raw = data_tags[3].text.strip() if data_tags[3] else "N/A"

            # Regex to extract Trails Open, Total Trails, and Percentage Open
            match = re.search(r"(\d+)\s*/\s*(\d+)(\d{2,3})?\s*(\d+%)?", open_trails_raw)

            if match:
                trails_open = int(match.group(1))
                total_trails_raw = match.group(2)
                last_digits = int(match.group(3)) if match.group(3) else "N/A"
                percentage_open = match.group(4) if match.group(4) else "N/A"

                # Correction logic
                total_trails = int(total_trails_raw)
                corrected_total_trails = total_trails

                # Attempt to correct by removing trailing digits
                for i in range(1, 4):  # Check removing 1 to 3 digits
                    possible_total = int(str(total_trails)[:-i]) if len(str(total_trails)) > i else total_trails

                    if possible_total >= trails_open:
                        if percentage_open != "N/A":
                            calculated_percentage = round((trails_open / possible_total) * 100)
                            if calculated_percentage == int(percentage_open.strip('%')):
                                corrected_total_trails = possible_total
                                break
                        else:
                            # If percentage is not provided, use logical deduction
                            if possible_total - trails_open <= 50:  # Assuming it's reasonable
                                corrected_total_trails = possible_total
                                break

                # Final validation
                if corrected_total_trails < trails_open:
                    corrected_total_trails = "N/A"

            else:
                trails_open, total_trails_raw, corrected_total_trails, percentage_open = "N/A", "N/A", "N/A", "N/A"

            snow_data.append({
                "Date": datetime.now().strftime("%Y-%m-%d"),
                "Resort": data_tags[0].text.strip() if data_tags[0] else "N/A",          # Resort Name
                "Snow (24hr)": data_tags[1].text.strip().rstrip('"').rstrip('-') if data_tags[1] else "N/A",      # Cleaned Snow (24hr)
                "Base Depth": data_tags[2].text.strip() if data_tags[2] else "N/A",       # Base Depth
                "Trails Open": trails_open,                                               # Cleaned Trails Open
                "Total Trails": corrected_total_trails,                                   # Renamed Corrected Total Trails
                "Lifts Open": data_tags[4].text.strip().rstrip('-') if data_tags[4] else "N/A"        # Cleaned Lifts Open
            })

    # Convert to DataFrame
    new_data_df = pd.DataFrame(snow_data)

    # Separate Base Depth into two columns
    new_data_df[['Base Depth', 'Base Depth Notes']] = new_data_df['Base Depth'].str.extract(r'(\d+\"(?:-\d+\")?)(.*)')

    # Calculate % Trails Open
    new_data_df["% Trails Open"] = (new_data_df["Trails Open"] / new_data_df["Total Trails"] * 100).round(2)

    # Append or create CSV
    if os.path.exists(csv_file_path):
        existing_data_df = pd.read_csv(csv_file_path)
        combined_df = pd.concat([existing_data_df, new_data_df], ignore_index=True)
    else:
        combined_df = new_data_df

    # Save combined data
    combined_df.to_csv(csv_file_path, index=False)
    logging.info(f"Data successfully saved to {csv_file_path}")

    # Display newly added data
    print(new_data_df)

except Exception as e:
    logging.error(f"An error occurred: {e}")
    print(f"An error occurred: {e}")


In [46]:
new_data_df_coloradoski.head(2)

,Date,Resort,Snow (24hr),Snow (48hr),Mid-Mt Depth,Surface Conditions,Lifts Open,Green Runs,Blue Runs,Black Runs,Double Black Runs
0,2025-02-09,Arapahoe Basin,"2""","8""",52”,HP/PP,9/9,100%,100%,100%,79%
1,2025-02-09,Aspen Highlands,"7""","11""",41”,PP,5/5,N/A,100%,100%,84%


In [49]:
new_data_df.head(2)

,Date,Resort,Snow (24hr),Base Depth,Trails Open,Total Trails,Lifts Open,Base Depth Notes,% Trails Open
0,2025-02-09,Arapahoe Basin Ski Area,"2""","52""",133,145,8/9,Variable Conditions,91.72
1,2025-02-09,Aspen Snowmass,"9""","53""",349,366,34/41,Powder,95.36


# MERGING THE TWO INTO ONE!

In [50]:
'''
We are in the position now where we have two CSV files. THere must be a way to now take these and extract the best numbers form this into a new dataframe.
'''


'\nWe are in the position now where we have two CSV files. THere must be a way to now take these and extract the best numbers form this into a new dataframe.\n'

In [2]:
import pandas as pd

In [ ]:
'''
Final Step --> Merge and Save as one final CSV
'''
import pandas as pd
import os
import shutil
from datetime import datetime

df_onthesnow = pd.read_csv(r"snow_report_data_onthesnow.csv")
df_coloradoski = pd.read_csv(r"snow_report_data_soloradoski.csv")

# Establish Mapping
resort_name_mapping = {
    "Arapahoe Basin Ski Area": "Arapahoe Basin",
    "Eldora Mountain Resort": "Eldora",
    "Aspen Snowmass": "Snowmass",
    "Monarch Mountain": "Monarch",
    "Purgatory": "Purgatory Resort",
    "Silverton Mountain": "Silverton"
}

# Rename the Resorts so they can all be joined
df_onthesnow['Resort'] = df_onthesnow['Resort'].replace(resort_name_mapping)

# Merge the two datasets
new_data = df_onthesnow.merge(right=df_coloradoski, how = "left", on = ['Resort', 'Date'])

# Define the final file path
final_csv_path = r"C:\Users\danie\OneDrive\Desktop\PersonalProjects\SkiWebScraping\Merged_SnowReportScraper.csv"

# Safety Check: Backup if the file exists
if os.path.exists(final_csv_path):
    # Create a backup with a timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_path = f"backup_snow_report_data_{timestamp}.csv"
    shutil.copy(final_csv_path, backup_path)
    print(f"Backup created at: {backup_path}")

    # Load the existing data
    existing_data = pd.read_csv(final_csv_path)
    
    # Append new data without creating duplicates
    combined_data = pd.concat([existing_data, new_data]).drop_duplicates(subset=['Resort', 'Date'], keep='last')
else:
    # No existing data, just use the new data
    combined_data = new_data

# Save the combined dataset
combined_data.to_csv(final_csv_path, index=False)

print(f"Final merged dataset saved as '{final_csv_path}' with backup safeguard.")


Final merged dataset saved as 'C:\Users\danie\OneDrive\Desktop\PersonalProjects\SkiWebScraping\Merged_SnowReportScraper.csv' with backup safeguard.


: 